In [2]:
from __future__ import annotations

import os
import re
import math
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Iterable, Literal
import requests
import numpy as np
import pandas as pd
from pandas import json_normalize
from bs4 import BeautifulSoup
from zeep import Client, helpers
import xmltodict
from utils import safe_serialize, listar_ops

In [3]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
periodos = ['2022-2026'] # <- Elegir períodos

### Detalles de votación

In [ ]:
def get_detalle(idx):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionDetalle(idx)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df
    
def get_vot_year(year):
    c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
    res = c.service.retornarVotacionesXAnno(year)
    if res:
        d = safe_serialize(res) or {}
        df = json_normalize(d)
        return df

In [ ]:
# === Config ===
PERIODO_RE = re.compile(r"^\s*(\d{4})\s*-\s*(\d{4})\s*$")

def normalize_col(name):
    name = name.strip()
    name = re.sub(r'[\s\.-]+', '_', name)          # espacios, puntos o guiones → _
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)  # camelCase → snake_case
    return name.lower()

def parse_periodo(periodo: str) -> tuple[int, int]:
    """
    Devuelve (inicio, fin) a partir de 'YYYY-YYYY'.
    Lanza ValueError si el formato no calza.
    """
    m = PERIODO_RE.match(str(periodo))
    if not m:
        raise ValueError(f"Formato de Periodo inválido: {periodo!r}")
    ini, fin = int(m.group(1)), int(m.group(2))
    if fin < ini:
        raise ValueError(f"Año final < inicial en Periodo: {periodo!r}")
    return ini, fin


def safe_concat(frames: Iterable[pd.DataFrame], **kwargs) -> pd.DataFrame:
    """
    Concatena ignorando None / DFs vacíos.
    """
    lst = [df for df in frames if df is not None and not df.empty]
    if not lst:
        return pd.DataFrame()
    return pd.concat(lst, ignore_index=True, **kwargs)

def build_detalle_periodo(nombre_periodo: str) -> pd.DataFrame:
    """
    Construye el detalle de votos para un periodo 'YYYY-YYYY':
      - junta votos por año
      - trae detalle por Id
      - explota Votos.Voto y normaliza
      - devuelve DataFrame final listo para guardar
    """
    inicio, fin = parse_periodo(nombre_periodo)

    # Incluimos ambos extremos del periodo (YYYY-YYYY)
    years = range(inicio, fin + 1)

    # 1) Votos por año
    df_vot_year = safe_concat([get_vot_year(y) for y in years])
    if df_vot_year.empty:
        return pd.DataFrame()

    if "Id" not in df_vot_year.columns:
        raise KeyError("La columna 'Id' no existe en df_vot_year.")

    # 2) Detalle por cada Id
    detalles = [get_detalle(idx) for idx in df_vot_year["Id"]]
    df_detalle = safe_concat(detalles)
    if df_detalle.empty:
        return pd.DataFrame()

    # 3) Explode y normalización
    col_votos = "Votos.Voto"
    if col_votos not in df_detalle.columns:
        # Nada que explotar; devolvemos el detalle tal cual
        return df_detalle.reset_index(drop=True)

    df_explode = df_detalle.explode(col_votos, ignore_index=True)

    # Algunas filas podrían no tener Votos.Voto (NaN) → normalizamos solo las no nulas
    votos_series = df_explode[col_votos]
    has_voto = votos_series.notna()

    if has_voto.any():
        # normalizamos únicamente las filas con dato
        df_votos_norm = json_normalize(votos_series[has_voto])
        # creamos un df vacío con el mismo índice para alinear
        df_votos_full = pd.DataFrame(index=df_explode.index)
        df_votos_full.loc[has_voto, df_votos_norm.columns] = df_votos_norm.values
        # concatenamos columnas normalizadas al explode original
        df_final = pd.concat([df_explode.drop(columns=[col_votos]), df_votos_full], axis=1)
    else:
        # No había votos anidados
        df_final = df_explode.drop(columns=[col_votos])
        
    to_drop = ['Votos', 'Diputado.Nombre2', 'Diputado.FechaNacimiento', 
           'Diputado.FechaDefucion', 'Diputado.RUT', 'Diputado.RUTDV',
          'Diputado.Sexo', 'Diputado.Militancias']
    
    df_final.drop(columns=to_drop, inplace=True)
    df_final.columns = [normalize_col(c) for c in df_final.columns]
    df_final = df_final[df_final["tipo__value_1"] == "Proyecto de Ley"]
    return df_final.reset_index(drop=True)

# === Loop principal por periodos ===
# Usamos itertuples para evitar el bug de 'rows/row' y ganar velocidad
for row in df_periodos.itertuples(index=False):
    nombre_periodo = getattr(row, "Periodo", None)
    if pd.isna(nombre_periodo):
        print("Saltando fila sin 'Periodo'.")
        continue

    try:
        df_out = build_detalle_periodo(str(nombre_periodo))
    except Exception as e:
        print(f"Error procesando periodo {nombre_periodo!r}: {e}")
        continue

    carpeta = DATA_DIR / sanitize(str(nombre_periodo))  # p.ej., data/1990-1994
    carpeta.mkdir(parents=True, exist_ok=True)

    out_path = carpeta / "detalle.csv"
    if df_out.empty:
        print(f"Periodo {nombre_periodo}: no se generaron filas. Archivo NO creado.")
        continue
    
    # Guardado prolijo
    df_out.to_csv(out_path, index=False, encoding="utf-8", lineterminator="\n")
    print(f"Guardado {out_path}  (filas: {len(df_out):,})")